 Spearman footrule : The sum of absolute values of the difference in the ranks.

**Reference**
1. An efficient approach for the rank aggregation problem : doi:10.1016/j.tcs.2006.05.024

**Rank Aggregation Problem**

1. Objects under cosideration are ordered according to several different criterion.

2. One looks for a ranking that is as close as possible to or, in a broader sense, combines—the rankings obtained in the first step.


**Implementation** 

1. We're given the rankings that are to be combined. We're also given the proposed ranking.

2. Use spearman footrule to get the distance between the proposed ranking and the given rankings.

3. The ranking that minimizes the sum of the distances from proposed ranking to all the given rankings is returned as the output of the rank aggregation problem.


**Assumptions**

1. We've a full list of ranking present i.e given 3 items we've a score for each of them aka no partial ranking.

2. If there are score collisions, all the items with same score are assigned the same rank (minimum of the group) and    an appropriate number of ranks are skipped e.g
 
Item  Score  Rank
 
 A : 100   : 1
 
 B : 100   : 1
 
 C : 100   : 1
 
 D : 65    : 4
 
 

**Interesting reads**

1. Kendall tau distance

2. http://www10.org/cdrom/papers/pdf/p577.pdf 
 

In [ ]:
import pandas as pd
import unittest

def scoresToRanks(scores):
    """returns a set of ranks based on the given scores for each measurement.
    :param score: dictionary of tuple where key is itemId and tuple contains scores by different metrics.

    :return: A pandas dataframe with ranks generated from each metric in a column.

    e.g scores = {'A' : (100,0.1),
          'B' : (90,0.3),
          'C' : (20, 0.2)}

        returns dataframe      Index     metric_0    metric_1
                               A            1           3
                               B            2           1
                               C            3           2

    """
    n_items = len(scores)
    if n_items:     # Check if scores is non-empty
        df = pd.DataFrame.from_dict(scores,orient='index')   # Using pandas to handle cases with huge number of entries
        df.columns = ['metric_'+ str(col) for col in df.columns]
        ranks = []
        for col in df.columns:
            df.sort_values(by=col,ascending=False,inplace=True) # Sort the index by the selected column
            df['rank'] = range(1,df.shape[0]+1) # Assign the ranks in descending order
            
            if df[col].nunique() != df.shape[0]: # check if all the scores are different i.e no collisions
                df[col] = df.groupby(col)['rank'].transform('min')
            else:
                df[col] = range(1,df.shape[0]+1)
                           
            df.drop('rank',axis=1,inplace=True)
    else:
        raise Exception("Sorry, no scores are provided.")
    return df


def sumSpearmanDistances(scores, proposedRank):
    """Calculates spearman footrule distance between the 
        proposed rank and the multiset of ranks generated by scores.
        
        :param scores: 
        :param proposedRank:
        
        :return:
        
        e.g ranks = [{'A':1, 'B':2, 'C':3},{'A':3, 'B':1, 'C':2}]
        
        proposedRank = {'A':1, 'B':2, 'C':3}  --> returns (0,4)
        proposedRank = {'A':2, 'B':3, 'C':1}  --> returns (4,4)
        proposedRank = {'A':3, 'B':1, 'C':2}  --> returns (4,0)
        proposedRank = {'A':1, 'B':3, 'C':2}  --> returns (2,4)
        proposedRank = {'A':2, 'B':1, 'C':3}  --> returns (2,2)
        proposedRank = {'A':3, 'B':2, 'C':1}  --> returns (4,2)
        """

    # ToDo : Add checks for partial rank cases and partial metric values being present.
    n_items = len(proposedRank)
    if n_items:
        ranks = scoresToRanks(scores) # Convert scores to ranks
        if not ranks.empty:
            ranks = ranks.reindex(proposedRank) # Reshuffle the dataset according to the proposed rank
            ranks['proposed_rank'] = range(1,ranks.shape[0]+1) # Assign proposed ranks
    
            distances = 0 
            for col in ranks.columns:
                if col != 'proposed_rank':
                    distances+=(ranks[col]-ranks['proposed_rank']).abs().sum() # calculate spearman footrule distance
            return distances    
 
    raise Exception("Sorry, proposed rank is not provided")
    


In [ ]:
class TestSumSpearmanDistances(unittest.TestCase):

    def test_case1(self):
        scores = {'A' : (100,0.1),'B' : (90,0.3),'C' : (20, 0.2)}
        proposedRank = ['A', 'B', 'C']
        self.assertEqual(sumSpearmanDistances(scores,proposedRank), 4)

    def test_case2(self):
        scores = {'A' : (100,0.1),'B' : (90,0.3),'C' : (20, 0.2)}
        proposedRank = ['C', 'A', 'B']
        self.assertEqual(sumSpearmanDistances(scores,proposedRank), 8)

    def test_case3(self):
        scores = {'A' : (100,0.1),'B' : (90,0.3),'C' : (20, 0.2)}
        proposedRank = ['A', 'C', 'B']
        self.assertEqual(sumSpearmanDistances(scores,proposedRank), 6)
    
    def test_case4(self):
        scores = {'A' : (100,0.1),'B' : (90,0.3),'C' : (20, 0.2)}
        proposedRank = ['C', 'B', 'A']
        self.assertEqual(sumSpearmanDistances(scores,proposedRank), 6)
    
    def test_case5(self):
        scores = {'A' : (100,0.1),'B' : (90,0.3),'C' : (20, 0.2)}
        proposedRank = ['B', 'A', 'C']
        self.assertEqual(sumSpearmanDistances(scores,proposedRank), 4)
    
    def test_case6(self):
        scores = {'A' : (100,0.1),'B' : (90,0.3),'C' : (20, 0.2)}
        proposedRank = ['B', 'C', 'A']
        self.assertEqual(sumSpearmanDistances(scores,proposedRank), 4)
        
    def test_rank_collisions(self):
        scores = {'A' : (100,0.1), 'B' : (90,0.3), 'C' : (20, 0.2),
                 'D' : (100,0.1), 'E' : (90,0.3)}
        proposedRank = ['A','B','C','D','E']
        self.assertEqual(sumSpearmanDistances(scores,proposedRank), 16)
    
    def test_raise_proposed_rank(self):
        scores = {'A' : (100,0.1),'B' : (90,0.3),'C' : (20, 0.2)}
        proposedRank = []
        with self.assertRaises(Exception) as err:
            sumSpearmanDistances(scores,proposedRank)
            self.assertTrue("Sorry, proposed rank is not provided" in err.exception.value)

    def test_raise_scores(self):
        scores = {}
        proposedRank = ['A','B','C']
        with self.assertRaises(Exception) as err:
            sumSpearmanDistances(scores,proposedRank)
            self.assertTrue("Sorry, no scores are provided." in err.exception.value)
            
unittest.main(argv=['ignored', '-v'], exit=False);